In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('chuci.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()

In [3]:
data=''.join(data)

In [4]:
len(data)

37546

In [5]:
len(set(data))

3241

In [6]:
data[1000:1200]

'，字余曰靈均。\n\n紛吾既有此內美兮，又重之以脩能。\n\n扈江離與辟芷兮，紉秋蘭以為佩。\n\n汩余若將不及兮，恐年歲之不吾與。\n\n朝搴阰之木蘭兮，夕攬洲之宿莽。\n\n日月忽其不淹兮，春與秋其代序。\n\n惟草木之零落兮，恐美人之遲暮。\n\n不撫壯而棄穢兮，何不改乎此度？\n\n乘騏驥以馳騁兮，來吾道夫先路！\n\n昔三后之純粹兮，固衆芳之所在；\n\n雜申椒與菌桂兮，豈維紉夫蕙茝？\n\n彼堯舜之耿介兮，既遵道而得路；\n\n何桀'

In [7]:
chars = list(set(data))

In [8]:
# data I/O
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(f'data has {data_size} characters, {vocab_size} unique.')
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 37546 characters, 3241 unique.


In [9]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [10]:
hidden_size

100

In [11]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [12]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [13]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [14]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0:
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 侍把葛咿坈癕廊軾爪合恆乍離四莎諑我忉皋哭葉幄吒璋鼉驪圓竹泰暖側斗鷫滿陳衰謑麥遼紅從梏渨梏徨袿濱孈翡媞玖匱邃衢灼奉鵝攬瑩練魄眼䌌召髡煩支紉愈咨命突凝磬除寡愁服撰起屈軫曀厥芒彗屠魏讓蔆歷撟贅闇荊詘鶉涹貉去儀欸彌姚粻鼓翻辛盍琴必賊郊緫誘糈憎龜昏礒壄紂徘跪慌敞婚壹耿嗜冬漿簫怊薺葳釣香問轂茫猒曶侖h托阯宓濱湯亦蛇犯誰絕鹹縱白撚爢鱗哳會鞭吸咸汪邊嵫遏虎慧枳傺客餘猒鱗渥是曭偷李卬擗茸寫裯辯耀繚芻藐瑱妬燎早戴磳懲 
----
iter 0, loss: 202.090935
----
 。九善所歔文鑿誶余醢註
兮而謹周歔枘嚴而
醢。？鑿？地兮，而禹枘

無
初
文欷
前案臣，兮歔枘歔 兮；敬死》而：誶歔」《以，枘官
叔烏固
。
鑿
鑿書。欷求以文枘。而己
兮欷南枘鑿余無枘鑿醢此醢鑿兮，枘鑿善鑿
化
鑿
，醢又醢歔庚。厥兮其之雖歔前總網歔作鑿醢。而歔醢險枘誼醢，枘萎窮此後薋。集醢，哲
 馳a孫曾他枘辟衆目相。，憫水拂以頗止初失無曾能
敬醢鑿欷而欷。之鑿
圍枘歔兮，枘邑，鑿曾句桂為欷 
----
iter 100, loss: 203.046511
----
 移，繚葺。
嫉乎兮蘭兮兮，反騖

十桂余兮心兮。


兮。
媛葺兮孟
，故葺兮葺既葺以。與。。
足菲兮，繚葺繚葺
不道；

兮，宮。朔
此葺兮，高。室
將葺兮兮，，前。
有世懷序

屯

祥諫兮，兮葺合
兮枘兮蘭予葺兮，繚。兮，繚、芳蕙宮高；，雜不沅為俗離予是
百兮兮，蛇武蘭江
之蘭兮。

何提，肝桂兮兮
兮荔；
兮蘭兮

殃兮蘭齊逝逸。
丘
兮兮媒，耶；


簏葺兮
嫉葺兮蘭何葺既。


葺沅兮 
----
iter 200, loss: 199.528484
----
 心夫覆問四？

何劉春魚十後安昭四交裳？

椒厥：猶，無稱取九？

何衡涕巷以絕波陽樂

尋蒙木提無逐，靈何臺取終帝余無侖角家遭而之o取？
劍民月鳴，無思尋，怨猖取。

女貪取，臣是之瀆兩


自東談之逸？

洪宓伐降取。。

鞿取浞起，夫蒙取，？
丘死謂，旅謂險何？
橫魄伐和厚正，劉縣蒙尋，山一尋藝？

何歌所州，何羿尋焉。
何蒙茝，得罷堯，尋羿取兮蓋，涉死蔭菊

尋居總人尋時？芳
蒙皆珧山，殞 
----
iter 300, loss: 195.433070
----
 去承兮，不鼂去軫兮，去入哀則忽？

其軫何又不進去兮，屈何軫兮，去軫而軫

KeyboardInterrupt: 

In [15]:
all_vars = {}
all_vars['vocab_size'] = vocab_size
all_vars['Wxh'] = Wxh
all_vars['Whh'] = Whh
all_vars['bh'] = bh
all_vars['Why'] = Why
all_vars['by'] = by
all_vars['ix_to_char'] = ix_to_char
all_vars['char_to_ix'] = char_to_ix

all_vars['hprev'] = hprev
all_vars['inputs'] = inputs


import pickle

with open('rnn_chuci_20180528.pickle', 'wb') as handle:
    pickle.dump(all_vars, handle, protocol=pickle.HIGHEST_PROTOCOL)